In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import OpenBlender
import json
import datetime

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
#pd.set_option('display.max_rows', 500)

In [91]:
#OpenBlender API credentials
apitoken = pd.read_csv("BlenderApi.csv")

In [92]:
#Normalize date to UNIX timestamp
token = apitoken["key"][0]
df = pd.read_csv("C:\\Users\\hp\\Desktop\\Cv_project\\NLP_Predict\\raw_btc\\raw_btc.csv")
df.sort_values('date',inplace = True)
df = df.reset_index(drop=True)
df["date"] = df["date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.%fZ"))
df["date"] = df["date"].apply(lambda x: x.timestamp())
df["date"] = df["date"].apply(lambda x: OpenBlender.unixToDate(x, timezone = 'GMT'))
df

,date,open,high,low,close,volume
0,02-01-2021 23:59:59,29376.455834,33155.118819,29091.181191,32127.267939,6.786542e+10
1,03-01-2021 23:59:59,32129.407356,34608.556738,32052.317076,32782.024466,7.866524e+10
2,04-01-2021 23:59:59,32810.947644,33440.218569,28722.755271,31971.913522,8.116348e+10
3,05-01-2021 23:59:59,31977.041132,34437.590592,30221.186739,33992.429344,6.754732e+10
4,06-01-2021 23:59:59,34013.614533,36879.698569,33514.033742,36824.364410,7.528943e+10
...,...,...,...,...,...,...
515,01-06-2022 23:59:59,31792.553809,31957.284317,29501.588523,29799.079714,4.113582e+10
516,02-06-2022 23:59:59,29794.890080,30604.734859,29652.704983,30467.487741,2.908356e+10
517,03-06-2022 23:59:59,30467.806840,30633.034476,29375.689302,29704.391357,2.617555e+10
518,04-06-2022 23:59:59,29706.139573,29930.563947,29500.006350,29832.914225,1.658837e+10


In [93]:
#Create change value (log_diff is an approximation to change value)
df["log_diff"] = np.log(df['close']) - np.log(df['open'])
df.head()

,date,open,high,low,close,volume,log_diff
0,02-01-2021 23:59:59,29376.455834,33155.118819,29091.181191,32127.267939,6.786542e+10,0.089512
1,03-01-2021 23:59:59,32129.407356,34608.556738,32052.317076,32782.024466,7.866524e+10,0.020109
2,04-01-2021 23:59:59,32810.947644,33440.218569,28722.755271,31971.913522,8.116348e+10,-0.025904
3,05-01-2021 23:59:59,31977.041132,34437.590592,30221.186739,33992.429344,6.754732e+10,0.061120
4,06-01-2021 23:59:59,34013.614533,36879.698569,33514.033742,36824.364410,7.528943e+10,0.079399


In [94]:
#Create Target data
df['target'] = [1 if log_diff >0 else 0 for log_diff in df['log_diff']]
df.head()

,date,open,high,low,close,volume,log_diff,target
0,02-01-2021 23:59:59,29376.455834,33155.118819,29091.181191,32127.267939,6.786542e+10,0.089512,1
1,03-01-2021 23:59:59,32129.407356,34608.556738,32052.317076,32782.024466,7.866524e+10,0.020109,1
2,04-01-2021 23:59:59,32810.947644,33440.218569,28722.755271,31971.913522,8.116348e+10,-0.025904,0
3,05-01-2021 23:59:59,31977.041132,34437.590592,30221.186739,33992.429344,6.754732e+10,0.061120,1
4,06-01-2021 23:59:59,34013.614533,36879.698569,33514.033742,36824.364410,7.528943e+10,0.079399,1


In [6]:
#Add timestamp to dataset and retrieve useful data
format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df['timestamp'] = OpenBlender.dateToUnix(df['date'], 
                                           date_format = format, 
                                           timezone = timezone)
df = df[['date', 'timestamp', 'close', 'target']]
df.head()

,date,timestamp,close,target
0,02-01-2021 23:59:59,1.609632e+09,32127.267939,1
1,03-01-2021 23:59:59,1.609718e+09,32782.024466,1
2,04-01-2021 23:59:59,1.609805e+09,31971.913522,0
3,05-01-2021 23:59:59,1.609891e+09,33992.429344,1
4,06-01-2021 23:59:59,1.609978e+09,36824.364410,1


In [7]:
#(Vizualisation sake) Search for a dataset to enrich our dataset due to poor correlation of other variables
search_keyword = 'bitcoin'
df = df.sort_values('timestamp').reset_index(drop = True)
print('From : ' + OpenBlender.unixToDate(min(df.timestamp)))
print('Until: ' + OpenBlender.unixToDate(max(df.timestamp)))
OpenBlender.searchTimeBlends(token,
                             df.timestamp,
                             search_keyword)

From : 02-01-2021 23:59:59
Until: 05-06-2022 23:59:59


[{'name': 'Bitcoins Cash Historical data',
  'url': 'https://www.openblender.io/#/dataset/explore/5d9f899b951629329a748593',
  'id_dataset': '5d9f899b951629329a748593',
  'features': ['close',
   'high',
   'low',
   'market_cap',
   'open',
   'timestamp',
   'volume'],
  'num_observations': 1808,
  'intersection': 100,
  'description': 'Daily prices and volumes for the digital currency BitcoinsCash.'},
 {'name': 'Bitcoin SV Price',
  'url': 'https://www.openblender.io/#/dataset/explore/5db7a7c99516294f3f6c2faf',
  'id_dataset': '5db7a7c99516294f3f6c2faf',
  'features': ['change',
   'high',
   'low',
   'open',
   'price',
   'timestamp',
   'volume'],
  'num_observations': 917,
  'intersection': 100,
  'description': 'Daily Price of Bitcoin SV'},
 {'name': 'Bitcoin Gold Price',
  'url': 'https://www.openblender.io/#/dataset/explore/5db7aa0b9516294f3f6c30ca',
  'id_dataset': '5db7aa0b9516294f3f6c30ca',
  'features': ['change',
   'high',
   'low',
   'open',
   'price',
   'timestamp

In [8]:
##Blend BTCNews tweet data to the data set
blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text'
            }


df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
                                 
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df.head()

Task ID: '629e208c0895fafb4a9d8d4c'.
Total estimated consumption: 2464.0 processing units.
Task confirmed. Starting download..
100%


,date,timestamp,close,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days
0,02-01-2021 23:59:59,1.609632e+09,32127.267939,1,14,[despite the investors criticism bitcoins mark...
1,03-01-2021 23:59:59,1.609718e+09,32782.024466,1,9,[on the 12th anniversary of the network coming...
2,04-01-2021 23:59:59,1.609805e+09,31971.913522,0,15,[kim dotcoms nextgeneration contentmonetizatio...
3,05-01-2021 23:59:59,1.609891e+09,33992.429344,1,16,[the volume in december of nft artworks sold w...
4,06-01-2021 23:59:59,1.609978e+09,36824.364410,1,23,[the scheme promised a monthly return of 2030 ...


In [9]:
#Create a positive and negative value from ngrams concerning 
#A positive/negative news for bitcoin and blend it with our data set
positive_filter = {'name' : 'positive', 
                   'match_ngrams': ['positive', 'buy', 
                                    'bull', 'boost','benefits','gain','rise','growth']}
blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text',
                'filter_text' : positive_filter
            }
df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)

negative_filter = {'name' : 'negative', 
                   'match_ngrams': ['negative', 'loss', 'drop', 'plummet', 'sell', 'fundraising','bear','luna']}
blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text',
                'filter_text' : negative_filter
            }
df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)

Task ID: '629e209e0895fafb4a9d8d4c'.
Total estimated consumption: 2464.0 processing units.
Task confirmed. Starting download..
100%
Task ID: '629e20a80895fafb4a9d8d4c'.
Total estimated consumption: 2464.0 processing units.
Task confirmed. Starting download..
100%


In [10]:
#As we can see the correlation is still quite small
df_anchor = df.copy(deep=True)
features = ['target', 'BITCOIN_NE.text_COUNT_last1days:positive', 'BITCOIN_NE.text_COUNT_last1days:negative']
df_anchor[features].corr()

,target,BITCOIN_NE.text_COUNT_last1days:positive,BITCOIN_NE.text_COUNT_last1days:negative
target,1.000000,0.003548,0.036056
BITCOIN_NE.text_COUNT_last1days:positive,0.003548,1.000000,0.117742
BITCOIN_NE.text_COUNT_last1days:negative,0.036056,0.117742,1.000000


In [11]:
##We need more metrics for our ML hence we blend a text vectorize to it
blend_source = { 
                'id_textVectorizer':'5f739fe7951629649472e167'
               }
df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df_anchor.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw').add_prefix('VEC.')
df_blend.drop(labels ="VEC.2020",axis=1,inplace=True)
df_blend.head()

Task ID: '629e20b10895fafb4a9d8d4c'.
Total estimated consumption: 2466.46 processing units.
Task confirmed. Starting download..
67.0%
100%


,VEC.block reward,VEC.shop,VEC.code,VEC.virgin,VEC.chain,VEC.miner,VEC.demand,VEC.dollar,VEC.month,VEC.funding debate,...,VEC.safe,VEC.heats,VEC.profits,VEC.volumes,VEC.time,VEC.laws,VEC.expands,VEC.integrates,VEC.digital gold,VEC.minister
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0


In [12]:
df_anchor = pd.concat([df_anchor, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df_anchor.head()

,date,timestamp,close,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,BITCOIN_NE.text_COUNT_last1days:positive,BITCOIN_NE.text_last1days:positive,BITCOIN_NE.text_last1days:negative,BITCOIN_NE.text_COUNT_last1days:negative,...,VEC.safe,VEC.heats,VEC.profits,VEC.volumes,VEC.time,VEC.laws,VEC.expands,VEC.integrates,VEC.digital gold,VEC.minister
0,02-01-2021 23:59:59,1.609632e+09,32127.267939,1,14,[despite the investors criticism bitcoins mark...,3,[if you buy something like bitcoin or some cry...,"[when we are all paid in bitcoin, no one can t...",3,...,0,0,0,0,0,0,0,0,0,0
1,03-01-2021 23:59:59,1.609718e+09,32782.024466,1,9,[on the 12th anniversary of the network coming...,2,[after microstrategy used a total of 1_125 bil...,[],0,...,0,0,0,0,0,0,0,0,0,0
2,04-01-2021 23:59:59,1.609805e+09,31971.913522,0,15,[kim dotcoms nextgeneration contentmonetizatio...,1,[cowen believes that governments will respond ...,[],0,...,0,0,0,0,0,0,0,0,0,0
3,05-01-2021 23:59:59,1.609891e+09,33992.429344,1,16,[the volume in december of nft artworks sold w...,2,"[the economist agrees that crypto assets, such...","[despite the reported decline, it will nonethe...",2,...,0,0,0,1,0,0,0,0,0,0
4,06-01-2021 23:59:59,1.609978e+09,36824.364410,1,23,[the scheme promised a monthly return of 2030 ...,1,[grayscale sells off all of its xrp holdings t...,[grayscale sells off all of its xrp holdings t...,3,...,0,0,0,1,0,0,0,0,1,0


In [173]:
##We shift by one line for the sake
df_anchor["target"] = df_anchor["target"].shift(-1)
df_anchor.dropna(inplace=True)
df_anchor.reset_index(inplace=True, drop=True)

In [189]:
#Safe Copy
df_final = df_anchor.copy(deep=True)

In [190]:
##Preparing the test and training data (while also removing redundant information)
corr_matrix = df_final.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
df_final.drop([column for column in upper.columns if any(upper[column] > 0.5)], axis=1, inplace=True)
div = int(round(len(X) * 0.29))

X = df_final.loc[:, df_final.columns != 'target'].select_dtypes(include=[np.number]).drop(labels="timestamp",axis=1).values
Y = df_final.loc[:,['target']].values

X_test = X[:div]
Y_test = Y[:div]

X_train = X[div:]
Y_train = Y[div:]

In [193]:
#Train the model
rf = RandomForestRegressor(n_estimators = 1500, bootstrap=True, random_state=42)
rf.fit(X_train, Y_train.ravel())
Y_pred = rf.predict(X_test)

In [194]:
##Get Confusion matrix and end value
df_res = pd.DataFrame({'y_test':Y_test.ravel(), 'y_pred':Y_pred.tolist()})
threshold = 0.5
preds = [1 if val > threshold else 0 for val in df_res['y_pred']]

print(confusion_matrix(preds, df_res['y_test']))
print('Accuracy Score:')
print(accuracy_score(preds, df_res['y_test']))
print('Precision Score:')
print(precision_score(preds, df_res['y_test']))

[[36 22]
 [41 44]]
Accuracy Score:
0.5594405594405595
Precision Score:
0.6666666666666666


In [140]:
df_anchor.to_csv("trainingdata.csv", index=False)